### Important 
This notebook is used to run code (like a main.py) __only__.

All methods must be written in separate .py files. 

Allowed here: Imports, data loading, method calling. 
 

Imports

In [1]:
import pandas as pd
import sklearn as sk

Constants

In [3]:
DATA_FOLDER = 'data/'
TWITTER_FOLDER = 'twitter-datasets/'

Data Loading

In [25]:
#Load partial datasets for now. 
#read_table automatically uses the new line as a separator. 
train_neg = pd.read_table(DATA_FOLDER+TWITTER_FOLDER+"train_neg.txt", header=None, names=['Tweet'], low_memory=False)
train_pos = pd.read_table(DATA_FOLDER+TWITTER_FOLDER+"train_pos.txt", header=None, names=['Tweet'], low_memory=False)

In [26]:
train_neg.head()

,Tweet
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...


In [27]:
train_pos.head()

,Tweet
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,<user> just put casper in a box ! looved the...
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...
